In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 35 kB/s 
     |████████████████████████████████| 198 kB 36.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=c9bfa638016da52ab2f5f8668f53175a11690680abc2f247563541b9bdd236fc
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [3]:
spark = (
    SparkSession.builder
      .master('local')
      .appName('dataframe_set')
      .config('spark.ui.port', '4050')
      .getOrCreate()
)

In [4]:
spark

In [5]:
dados = [
             ("Anderson", "Vendas", "SP", 1500.00, 34, 1000.00),
             ("Kennedy", "Vendas", "CE", 1200.00, 56, 2000.00),
             ("Bruno", "Vendas", "SP", 1200.00, 30, 2300.00),
             ("Maria", "Financas", "CE", 3600.00, 24, 2300.00),
             ("Eduardo", "Financas", "CE", 4500.00, 40, 2400.00),
             ("Mendes", "Financas","RS", 8000.00, 36, 1900.00),
             ("Kethlyn", "Financas", "RS", 1200.00, 53, 1500.00),
             ("Thiago", "Marketing", "GO", 1100.00, 25, 1800.00),
             ("Carla", "Marketing", "GO", 2600.00, 50, 2100.00)
]

esquema = ["nome", "departamento", "estado", "salario", "idade", "bonus"]

df = spark.createDataFrame(data=dados, schema=esquema)
df.show()
df.printSchema()

+--------+------------+------+-------+-----+------+
|    nome|departamento|estado|salario|idade| bonus|
+--------+------------+------+-------+-----+------+
|Anderson|      Vendas|    SP| 1500.0|   34|1000.0|
| Kennedy|      Vendas|    CE| 1200.0|   56|2000.0|
|   Bruno|      Vendas|    SP| 1200.0|   30|2300.0|
|   Maria|    Financas|    CE| 3600.0|   24|2300.0|
| Eduardo|    Financas|    CE| 4500.0|   40|2400.0|
|  Mendes|    Financas|    RS| 8000.0|   36|1900.0|
| Kethlyn|    Financas|    RS| 1200.0|   53|1500.0|
|  Thiago|   Marketing|    GO| 1100.0|   25|1800.0|
|   Carla|   Marketing|    GO| 2600.0|   50|2100.0|
+--------+------------+------+-------+-----+------+

root
 |-- nome: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- salario: double (nullable = true)
 |-- idade: long (nullable = true)
 |-- bonus: double (nullable = true)



In [17]:
#Função de soma

df.groupBy(F.col("departamento")).sum("salario").show() 
  #Agrupa dos departamentos e soma os salários por departamento

+------------+------------+
|departamento|sum(salario)|
+------------+------------+
|      Vendas|      3900.0|
|   Marketing|      3700.0|
|    Financas|     17300.0|
+------------+------------+



In [20]:
#Função de contagem

df.groupBy(F.col("departamento")).count().show()
  #Agrupa dos departamentos e conta pessoas por departamento

+------------+-----+
|departamento|count|
+------------+-----+
|      Vendas|    3|
|   Marketing|    2|
|    Financas|    4|
+------------+-----+



In [26]:
#Função de Agregate (permite agregar mais de uma função)

display(df.groupBy(F.col("departamento")).agg(
    F.sum("salario").alias("somaSalarios"),
    F.mean("salario").alias("mediaSalario"),
    F.avg("salario").alias("mediaSalario2"),
    F.sum("bonus").alias("somaBonus"),
    F.max("bonus").alias("maxBonus"),
    F.min("bonus").alias("minSalario")
).show()) #Agrupa por departamentos a soma e media dos salarios, soma dos bonus e valores minimos e maximos dos mesmos.

+------------+------------+------------+-------------+---------+--------+----------+
|departamento|somaSalarios|mediaSalario|mediaSalario2|somaBonus|maxBonus|minSalario|
+------------+------------+------------+-------------+---------+--------+----------+
|      Vendas|      3900.0|      1300.0|       1300.0|   5300.0|  2300.0|    1000.0|
|   Marketing|      3700.0|      1850.0|       1850.0|   3900.0|  2100.0|    1800.0|
|    Financas|     17300.0|      4325.0|       4325.0|   8100.0|  2400.0|    1500.0|
+------------+------------+------------+-------------+---------+--------+----------+



None

In [27]:
#Função de Agregate (permite agregar mais de uma função)

(df.groupBy(F.col("departamento")).agg(
    F.sum("salario").alias("somaSalarios"),
    F.mean("salario").alias("mediaSalario"),
    F.avg("salario").alias("mediaSalario2"),
    F.sum("bonus").alias("somaBonus"),
    F.max("bonus").alias("maxBonus"),
    F.min("bonus").alias("minSalario")
    ).where(F.col("somaSalarios") >= 3000).show())

#Agrupa por departamentos a soma e media dos salarios, soma dos bonus e valores minimos e maximos dos mesmos,
#onde a soma dos salários sejam iguais ou maiores à 3000.

+------------+------------+------------+-------------+---------+--------+----------+
|departamento|somaSalarios|mediaSalario|mediaSalario2|somaBonus|maxBonus|minSalario|
+------------+------------+------------+-------------+---------+--------+----------+
|      Vendas|      3900.0|      1300.0|       1300.0|   5300.0|  2300.0|    1000.0|
|   Marketing|      3700.0|      1850.0|       1850.0|   3900.0|  2100.0|    1800.0|
|    Financas|     17300.0|      4325.0|       4325.0|   8100.0|  2400.0|    1500.0|
+------------+------------+------------+-------------+---------+--------+----------+

